In [88]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings("ignore")
import pickle
import os


import logging
from sklearn.model_selection import KFold, cross_val_score

In [89]:
data77 = pd.read_csv(r"C:\Users\ravit\Desktop\final one\Dataset\German_credit.csv")
data77.head()

,status,duration,credit_history,purpose,amount,savings,employment_duration,installment_rate,personal_status_sex,other_debtors,...,property,age,other_installment_plans,housing,number_credits,job,people_liable,telephone,foreign_worker,credit_risk
0,1,18,4,2,1049,1,2,4,2,1,...,2,21,3,1,1,3,2,1,2,1
1,1,9,4,0,2799,1,3,2,3,1,...,1,36,3,1,2,3,1,1,2,1
2,2,12,2,9,841,2,4,2,2,1,...,1,23,3,1,1,2,2,1,2,1
3,1,12,4,0,2122,1,3,3,3,1,...,1,39,3,1,2,2,1,1,1,1
4,1,12,4,0,2171,1,3,4,3,1,...,2,38,1,2,2,2,2,1,1,1


In [90]:
# Create a logs directory if it doesn't exist
log_dir = 'logger'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Configure the logger
logging.basicConfig(
    filename=os.path.join(log_dir, 'app.log'),
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

logger = logging.getLogger(__name__)


In [91]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   status                   1000 non-null   int64
 1   duration                 1000 non-null   int64
 2   credit_history           1000 non-null   int64
 3   purpose                  1000 non-null   int64
 4   amount                   1000 non-null   int64
 5   savings                  1000 non-null   int64
 6   employment_duration      1000 non-null   int64
 7   installment_rate         1000 non-null   int64
 8   personal_status_sex      1000 non-null   int64
 9   other_debtors            1000 non-null   int64
 10  present_residence        1000 non-null   int64
 11  property                 1000 non-null   int64
 12  age                      1000 non-null   int64
 13  other_installment_plans  1000 non-null   int64
 14  housing                  1000 non-null   int64
 15  numbe

In [92]:
data.columns

Index(['status', 'duration', 'credit_history', 'purpose', 'amount', 'savings',
       'employment_duration', 'installment_rate', 'personal_status_sex',
       'other_debtors', 'present_residence', 'property', 'age',
       'other_installment_plans', 'housing', 'number_credits', 'job',
       'people_liable', 'telephone', 'foreign_worker', 'credit_risk'],
      dtype='object')

In [93]:
logger.info('File loaded')

In [94]:
data.columns

Index(['status', 'duration', 'credit_history', 'purpose', 'amount', 'savings',
       'employment_duration', 'installment_rate', 'personal_status_sex',
       'other_debtors', 'present_residence', 'property', 'age',
       'other_installment_plans', 'housing', 'number_credits', 'job',
       'people_liable', 'telephone', 'foreign_worker', 'credit_risk'],
      dtype='object')

In [95]:
#Change to english if neccessary, if already done you can ignore
data=data.rename(columns={'laufkont':'status', 'laufzeit':'duration', 'moral':'credit_history', 'verw':'purpose',
                      'hoehe':'amount', 'sparkont':'savings', 'beszeit':'employment_duration', 'rate':'installment_rate',
                      'famges':'personal_status_sex', 'buerge':'other_debtors', 'wohnzeit':'present_residence',
                      'verm':'property', 'alter':'age', 'weitkred':'other_installment_plans', 'wohn':'housing',
                      'bishkred':'number_credits', 'beruf':'job', 'pers':'people_liable', 'telef':'telephone',
                      'gastarb':'foreign_worker', 'kredit':'credit_risk'})

In [96]:
data.info

<bound method DataFrame.info of      status  duration  credit_history  purpose  amount  savings  \
0         1        18               4        2    1049        1   
1         1         9               4        0    2799        1   
2         2        12               2        9     841        2   
3         1        12               4        0    2122        1   
4         1        12               4        0    2171        1   
..      ...       ...             ...      ...     ...      ...   
995       1        24               2        3    1987        1   
996       1        24               2        0    2303        1   
997       4        21               4        0   12680        5   
998       2        12               2        3    6468        5   
999       1        30               2        2    6350        5   

     employment_duration  installment_rate  personal_status_sex  \
0                      2                 4                    2   
1                      3     

In [97]:
data.head()

,status,duration,credit_history,purpose,amount,savings,employment_duration,installment_rate,personal_status_sex,other_debtors,...,property,age,other_installment_plans,housing,number_credits,job,people_liable,telephone,foreign_worker,credit_risk
0,1,18,4,2,1049,1,2,4,2,1,...,2,21,3,1,1,3,2,1,2,1
1,1,9,4,0,2799,1,3,2,3,1,...,1,36,3,1,2,3,1,1,2,1
2,2,12,2,9,841,2,4,2,2,1,...,1,23,3,1,1,2,2,1,2,1
3,1,12,4,0,2122,1,3,3,3,1,...,1,39,3,1,2,2,1,1,1,1
4,1,12,4,0,2171,1,3,4,3,1,...,2,38,1,2,2,2,2,1,1,1


In [98]:

# Ensure 'credit_risk' column exists
if 'credit_risk' not in data.columns:
    raise ValueError("The 'credit_risk' column is not found in the dataset. Please check the dataset.")

# Encode categorical variables
label_encoders = {}
for column in data.columns:
    if data[column].dtype == 'object':
        le = LabelEncoder()
        data[column] = le.fit_transform(data[column])
        label_encoders[column] = le

# Split the dataset

X = data.drop('credit_risk', axis=1)
y = data['credit_risk']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the dataset
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define models and hyperparameters for tuning
models = {
    'RandomForest': (RandomForestClassifier(random_state=42), {
        'n_estimators': [100, 200],
        'max_depth': [None, 10, 20]
    }),
    'GradientBoosting': (GradientBoostingClassifier(random_state=42), {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1, 1]
    }),
    'AdaBoost': (AdaBoostClassifier(random_state=42), {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 1]
    }),
    'SupportVectorMachine': (SVC(probability=True, random_state=42), {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf']
    })
}

# Find the best model using GridSearchCV
best_model = None
best_accuracy = 0

for model_name, (model, params) in models.items():
    print(f"Training and tuning {model_name}...")
    
    # Perform GridSearchCV with cross-validation
    grid_search = GridSearchCV(model, params, cv=5, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    
    # Best model from grid search
    best_model_from_search = grid_search.best_estimator_
    
    # Evaluate the best model on the test set
    y_pred = best_model_from_search.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{model_name} Model Accuracy after Grid Search: {accuracy:.2f}")
    
    # Save the best model if it outperforms previous models
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = best_model_from_search

# Check for class imbalance and model performance
print("\nClassification Report:")
print(classification_report(y_test, best_model.predict(X_test)))

# Check for confusion matrix to understand misclassification
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, best_model.predict(X_test)))

# Create and evaluate an ensemble model
if best_model is not None:
    print("Training ensemble model...")
    
    # Define an ensemble of all the tuned models
    ensemble = VotingClassifier(estimators=[
        ('rf', RandomForestClassifier(n_estimators=200, max_depth=20, random_state=42)),
        ('gb', GradientBoostingClassifier(n_estimators=200, learning_rate=0.1, random_state=42)),
        ('ab', AdaBoostClassifier(n_estimators=200, learning_rate=0.1, random_state=42)),
        ('svm', SVC(C=1, kernel='rbf', probability=True, random_state=42))
    ], voting='soft')
    
    # Train and evaluate the ensemble model
    ensemble.fit(X_train, y_train)
    y_pred = ensemble.predict(X_test)
    ensemble_accuracy = accuracy_score(y_test, y_pred)
    print(f"Ensemble Model Accuracy: {ensemble_accuracy:.2f}")
    
    # Use the ensemble model if it performs better
    if ensemble_accuracy > best_accuracy:
        best_accuracy = ensemble_accuracy
        best_model = ensemble


Training and tuning RandomForest...
RandomForest Model Accuracy after Grid Search: 0.76
Training and tuning GradientBoosting...
GradientBoosting Model Accuracy after Grid Search: 0.73
Training and tuning AdaBoost...
AdaBoost Model Accuracy after Grid Search: 0.74
Training and tuning SupportVectorMachine...
SupportVectorMachine Model Accuracy after Grid Search: 0.76

Classification Report:
              precision    recall  f1-score   support

           0       0.66      0.47      0.55        62
           1       0.79      0.89      0.84       138

    accuracy                           0.76       200
   macro avg       0.72      0.68      0.69       200
weighted avg       0.75      0.76      0.75       200


Confusion Matrix:
[[ 29  33]
 [ 15 123]]
Training ensemble model...
Ensemble Model Accuracy: 0.76


In [99]:
logger.info('ML models building completed')

In [100]:
with open('best_credit_risk_model.pkl', 'wb') as file:
    pickle.dump(best_model, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

with open('label_encoders.pkl', 'wb') as file:
    pickle.dump(label_encoders, file)

print(f"Best Model: {best_model.__class__.__name__} with Accuracy: {best_accuracy:.2f}")

Best Model: RandomForestClassifier with Accuracy: 0.76


In [101]:
logger.info('Pickling files completed')